In [ ]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
from google.colab import files
files.upload()

In [ ]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d aunanya875/suicidal-tweet-detection-dataset
!chmod 600 /root/.kaggle/kaggle.json

###El proposito de este programa es usar el modelo de ML de naive bayes para poder crear un modelo de queda la potencialidad de comerter suicidio babado en los tweets. El modelo usao la base de datos que acontinuacion se describe.

###Este conjunto de datos proporciona una colección de tweets junto con una anotación que indica si cada tweet está relacionado con el suicidio o no. El objetivo principal de este conjunto de datos es facilitar el desarrollo y la evaluación de modelos de aprendizaje automático para la clasificación de tweets que expresan sentimientos suicidas o no.

###Columnas:

###Tweet: Esta columna contiene el contenido textual de los tweets obtenidos de diversas fuentes. Los tweets cubren una amplia gama de temas, emociones y expresiones.
###Suicidio: Esta columna proporciona anotaciones que indican la clasificación de los tweets. Los valores posibles son:
###Publicación no suicida: esta etiqueta se asigna a tweets que no expresan ningún sentimiento o intención suicida.
###Publicación potencial de suicidio: esta etiqueta se asigna a tweets que muestran indicios de pensamientos, sentimientos o intenciones suicidas.



In [616]:
import pandas as pd
import numpy as np
import zipfile

In [617]:
with zipfile.ZipFile("/content/suicidal-tweet-detection-dataset.zip", "r" ) as zip_ref:
  zip_ref.extractall("/content/")

In [618]:
df = pd.read_csv("/content/Suicide_Ideation_Dataset(Twitter-based).csv")

In [619]:
df.shape

(1787, 2)

In [620]:
df.head()

,Tweet,Suicide
0,making some lunch,Not Suicide post
1,@Alexia You want his money.,Not Suicide post
2,@dizzyhrvy that crap took me forever to put to...,Potential Suicide post
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,Not Suicide post
4,Trying out &quot;Delicious Library 2&quot; wit...,Not Suicide post


In [621]:
data_2=df.copy()

In [622]:
data_2.isnull().sum()

Tweet      2
Suicide    0
dtype: int64

In [623]:
data_2 = data_2.dropna()
data_2.loc[:, 'Tweet'] = data_2['Tweet'].astype(str)

In [624]:
data_2.isnull().sum()

Tweet      0
Suicide    0
dtype: int64

In [625]:
data_2["Suicide"].replace("Not Suicide post", 0, inplace = True)
data_2["Suicide"].replace("Potential Suicide post ", 1, inplace = True)

In [626]:
data_2["Suicide"].unique()

array([0, 1])

In [627]:
data_2

,Tweet,Suicide
0,making some lunch,0
1,@Alexia You want his money.,0
2,@dizzyhrvy that crap took me forever to put to...,1
3,@jnaylor #kiwitweets Hey Jer! Since when did y...,0
4,Trying out &quot;Delicious Library 2&quot; wit...,0
...,...,...
1782,i have forgotten how much i love my Nokia N95-1,0
1783,Starting my day out with a positive attitude! ...,0
1784,"@belledame222 Hey, it's 5 am...give a girl som...",0
1785,2 drunken besties stumble into my room and we ...,0


In [630]:
!pip install demoji

In [631]:
import demoji
demoji.download_codes()

<ipython-input-631-eb011a9810ad>:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [632]:
def eliminar_emo(texto):
  return demoji.replace(texto,'')

In [672]:
data_2["Tweet"]=data_2["Tweet"].apply(eliminar_emo)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'[³áµ¹¸·â’Êæ±ËïÃ¯Â½ð­­­ª¾¶º¤¼´ó§¢¥§¼]', '', regex=True)
data_2["Tweet"] = data_2["Tweet"].str.replace(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'\[|\]', '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'[¬*"°¿^|£~–=¦¡/()%$.,!?@&#:;_\\\-+]', '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'\"|\`', '', regex=True)
data_2['Tweet'] = data_2['Tweet'].str.replace(r'\"|\'', '', regex=True)

In [673]:
data_2.head()

,Tweet,Suicide
0,making some lunch,0
1,Alexia You want his money,0
2,dizzyhrvy that crap took me forever to put tog...,1
3,jnaylor kiwitweets Hey Jer Since when did you ...,0
4,Trying out quotDelicious Library 2quot with mi...,0


In [674]:
import re
def buscar_no_letras(texto):
    no_letras_pattern = re.compile(r'[^a-zA-Z]')
    caracteres_no_letras = no_letras_pattern.findall(texto)
    return caracteres_no_letras

In [675]:
pd.DataFrame(data_2["Tweet"].apply(buscar_no_letras))

,Tweet
0,"[ , ]"
1,"[ , , , ]"
2,"[ , , , , , , , , , , , , , , ]"
3,"[ , , , , , , , , ]"
4,"[ , , , , 2, , , , , , , , , , , ..."
...,...
1782,"[ , , , , , , , , , 9, 5, 1]"
1783,"[ , , , , , , , , , , , ]"
1784,"[2, 2, 2, , , , 5, , , , , , , ]"
1785,"[2, , , , , , , , , , , , , , , ..."


In [676]:
from sklearn.feature_extraction.text import TfidfVectorizer
Danni_Flow = TfidfVectorizer()
V_TWEET = Danni_Flow.fit_transform(data_2['Tweet'])

In [677]:
P_F= pd.DataFrame(V_TWEET.toarray(), columns=Danni_Flow.get_feature_names_out())

In [678]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

In [679]:
X_train, X_test, Y_train, Y_test=train_test_split(
    P_F, data_2["Suicide"], test_size=0.3, random_state=42
)

In [680]:
nm_class=MultinomialNB()
nm_class.fit(X_train,Y_train)

MultinomialNB()

In [681]:
Y_Pred=nm_class.predict(X_test)

In [682]:
accuracy = accuracy_score(Y_test, Y_Pred)
report = classification_report(Y_test, Y_Pred)
print(f"Exactitud: {accuracy:.2f}")
print("Reporte de clasificación:\n", report)

Exactitud: 0.91
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.89      0.98      0.93       338
           1       0.96      0.80      0.87       198

    accuracy                           0.91       536
   macro avg       0.92      0.89      0.90       536
weighted avg       0.92      0.91      0.91       536



In [683]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(MultinomialNB(), P_F, data_2["Suicide"], scoring="accuracy", cv=10)

mean_score = scores.mean()
std_score = scores.std()
print(scores)
print("Media de puntajes:", mean_score)
print("Desviación estándar de puntajes:", std_score)

[0.88268156 0.91620112 0.93854749 0.87709497 0.93296089 0.91011236
 0.93258427 0.92134831 0.91011236 0.91011236]
Media de puntajes: 0.9131755696440902
Desviación estándar de puntajes: 0.019358931235342106


In [684]:
prediccion_X=pd.DataFrame({
    "Y_pred" : nm_class.predict(P_F),
    "Y_original" : data_2["Suicide"]
})

In [685]:
Falsos_neg=prediccion_X[(prediccion_X['Y_original']==1) & (prediccion_X['Y_pred']==0)]
Falsos_pos=prediccion_X[(prediccion_X['Y_original']==0) & (prediccion_X['Y_pred']==1)]

In [686]:
Falsos_pos.sum()

Y_pred        8
Y_original    0
dtype: int64

In [687]:
Falsos_neg.sum()

Y_pred         0
Y_original    54
dtype: int64